# Chargement du modèle GoogLeNet pré-entraîné

 charge le modèle **GoogLeNet** pré-entraîné sur le dataset ImageNet, en utilisant la bibliothèque PyTorch.

### Imports nécessaires

- **`torch`** : Bibliothèque principale de PyTorch pour le deep learning
- **`torch.nn`** : Module contenant les composants pour construire des réseaux de neurones
- **`torchvision.models`** : Module contenant des architectures de modèles pré-entraînés

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.googlenet(pretrained=True)

